# Time taken to complete degree
Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both Bachelor semestre 1 and Bachelor semestre 6. Compute how many months it took each student to go from the first to the sixth semester. Partition the data between male and female students, and compute the average -- is the difference in average statistically significant?

In [2]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup as bs

#Add the base url where the form is
base_url = "http://isa.epfl.ch/imoniteur_ISAP/!gedpublicreports.htm?ww_i_reportmodel=133685247"

Applying the filters to get the Bachelor semester 1 in informatique in 2015-16 gives the following url: 
```
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=&ww_x_UNITE_ACAD=249847&zz_x_PERIODE_ACAD=&ww_x_PERIODE_ACAD=213638028&zz_x_PERIODE_PEDAGO=&ww_x_PERIODE_PEDAGO=249108&zz_x_HIVERETE=&ww_x_HIVERETE=2936286&dummy=ok
```
Feeding this url to postman interceptor gives us the following parameter values:
```
ww_b_list:1
ww_i_reportmodel:133685247
ww_c_langue:
ww_i_reportModelXsl:133685270
zz_x_UNITE_ACAD:
ww_x_UNITE_ACAD:249847
zz_x_PERIODE_ACAD:
ww_x_PERIODE_ACAD:213638028
zz_x_PERIODE_PEDAGO:
ww_x_PERIODE_PEDAGO:249108
zz_x_HIVERETE:
ww_x_HIVERETE:2936286
dummy:ok
```
We used inspect element  to get the url of the page that displays only the data table without the form. The url was as follows:
```
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=1897032870&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=213638028&ww_x_PERIODE_PEDAGO=249108&ww_x_HIVERETE=2936286
```
Feeding this url to postman interceptor gives us the following parameter values:
```
ww_x_GPS:1897032870
ww_i_reportModel:133685247
ww_i_reportModelXsl:133685270
ww_x_UNITE_ACAD:249847
ww_x_PERIODE_ACAD:213638028
ww_x_PERIODE_PEDAGO:249108
ww_x_HIVERETE:2936286
```
Looking at the HTML code, we can figure out what each of the parameters stans for:
**ww_x_GPS**: 